In [2]:
%matplotlib inline
import torch
import random

首先构造参数为w和b以及带有一个噪声项$\epsilon$的人造数据集**y** = **X** **w** + b + $\epsilon$

In [7]:
def synthentic_data(w, b, num_samples):
    X = torch.normal(0, 1, (num_samples, len(w)))  ##X为均值为0方差为1的随机数，数量和参数w的数量一致
    y = torch.matmul(X, w) + b 
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape(-1, 1) ##将x和y的形状都变为列向量
    
w_true = torch.tensor([2, -3.4])
b_true = 4.2

features, labels = synthentic_data(w_true, b_true, 1000)

In [9]:
print('features:', features[0], '\nlabel:', labels[0])

features: tensor([-0.8105, -0.3568]) 
label: tensor([3.8019])


构造一个函数能够每次随机小批量地从数据集中采样

In [12]:
def data_batch(batch_size, feature, label):
    num_example = len(feature)
    induice = list(range(num_example))
    random.shuffle(induice)
    
    for i in range(0, num_example, batch_size):
        batch_induice = torch.tensor(induice[i:min(i + batch_size,num_example)])
        yield feature[batch_induice], label[batch_induice]
        
batch_size = 10
for X, y in data_batch(batch_size, features, labels):
    print (X, '\n', y)
    break

tensor([[ 0.4418, -0.2143],
        [-1.8064,  0.6571],
        [ 0.7412,  0.1856],
        [ 0.6559,  0.8517],
        [-0.6508, -0.8636],
        [-0.2165,  0.0563],
        [-0.1913, -0.6041],
        [-2.3457,  1.4957],
        [-1.1084, -1.6875],
        [ 0.9823,  0.6570]]) 
 tensor([[ 5.8165],
        [-1.6394],
        [ 5.0659],
        [ 2.6219],
        [ 5.8414],
        [ 3.5669],
        [ 5.8631],
        [-5.5850],
        [ 7.7229],
        [ 3.9221]])


初始化模型参数w和b

In [16]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

构造线性模型

In [17]:
def linear_reg(X, w, b):
    return torch.matmul(X, w) + b

定义损失函数

In [21]:
##损失函数为MSE
def loss_func(y, y_hat):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

定义优化器

In [19]:
##优化器为随机梯度下降SGD
def SGD(params, learning_rate, batch_size):
    with torch.no_grad():
        for param in params:
            param -= learning_rate * param.grad / batch_size
            param.grad.zero_()

In [22]:
learning_rate = 0.03
net = linear_reg
loss = loss_func
epochs = 10

for epoch in range(epochs):
    for X,y in data_batch(batch_size, features, labels):
        loss = loss_func(net(X, w, b), y)
        loss.sum().backward()
        SGD([w,b], learning_rate, batch_size)
    
    with torch.no_grad():
        train_loss = loss_func(net(features,w,b), labels)
        print (f'epoch { epoch + 1 }, loss {float(train_loss.mean()):f}')

epoch 1, loss 0.055980
epoch 2, loss 0.000252
epoch 3, loss 0.000050
epoch 4, loss 0.000049
epoch 5, loss 0.000049
epoch 6, loss 0.000049
epoch 7, loss 0.000049
epoch 8, loss 0.000049
epoch 9, loss 0.000049
epoch 10, loss 0.000049


使用pytorch设定好的内置函数实现

In [27]:
from torch.utils import data
import numpy as np

构造自定义数据集

In [32]:
def Dataset(Data, batch_size, is_train=True):
    dataset = data.TensorDataset(*Data)
    dataloader = data.DataLoader(dataset, batch_size, shuffle=is_train)
    return dataloader

batch_size = 10
DataSet = Dataset((features, labels), batch_size)

next(iter(DataSet))

[tensor([[-1.6186,  1.1914],
         [-0.7390,  0.7205],
         [ 0.9826,  0.6103],
         [ 0.8132, -0.0249],
         [-0.4938,  0.8550],
         [-0.0217,  0.4927],
         [ 0.8233,  0.3651],
         [ 0.3465, -0.4650],
         [ 0.0432, -0.1148],
         [ 0.4177,  0.7377]]),
 tensor([[-3.0947],
         [ 0.2740],
         [ 4.0838],
         [ 5.9232],
         [ 0.3082],
         [ 2.4810],
         [ 4.6042],
         [ 6.4671],
         [ 4.6845],
         [ 2.5223]])]

定义网络

In [37]:
import torch.nn as nn

reg = nn.Sequential(nn.Linear(2,1))

初始化权重

In [35]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

定义损失函数

In [36]:
Loss = nn.MSELoss()

构造优化器

In [38]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.03)

In [40]:
num_epochs = 3

for epoch in range(num_epochs):
    for X, y in DataSet:
        train_loss = Loss(net(X), y)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
    train_loss = Loss(net(features), labels)
    print(f' epoch {epoch + 1}, Loss {train_loss:f}')

 epoch 1, Loss 0.000420
 epoch 2, Loss 0.000099
 epoch 3, Loss 0.000099
